In [1]:
from dataclasses import dataclass, asdict
import requests
import time
import duckdb
import pandas as pd
from configparser import ConfigParser
import pgeocode

%load_ext sql
%config SqlMagic.displaylimit = 10

Deploy Panel apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [2]:
config = ConfigParser()
config.read("../../.config")

['../../.config']

In [3]:
api_key = config['GoogleMaps']['API_KEY']
base_url = config['GoogleMaps']['base_url']
radius = 50000  # in meters
place_type = 'shopping'  # Example place type
address = 'Luxembourg City, Luxembourg'


In [12]:
def format_coordinates(latitude, longitude):
    return f"{latitude},{longitude}"

def get_coordinates(api_key, address):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            print(f"Error in response: {data['status']}")
            return None, None
    else:
        print(f"HTTP error: {response.status_code}")
        return None, None




In [ ]:

latitude, longitude = get_coordinates(API_KEY, address)

if latitude and longitude:
    formatted_location = format_coordinates(latitude, longitude)
    # print(formatted_location)  # Output will be the coordinates of Luxembourg City
else:
    print("Could not retrieve coordinates.")

In [ ]:
# url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={formatted_location}&radius={radius}&key={API_KEY}"
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={formatted_location}&radius={radius}&type={place_type}&key={API_KEY}"

response = requests.get(url)
places = response.json()

for place in places['results']:
    print(f"Name: {place['name']}")
    print(f"Address: {place.get('vicinity', 'N/A')}")
    print(f"Place ID: {place['place_id']}")
    print('---')


In [14]:
place_id = 'ChIJ3e9sj9VIlUcRUU6bnYZ2tcE'  # Example place ID

details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={API_KEY}"

details_response = requests.get(details_url)
place_details = details_response.json()


print(f"Name: {place_details['result']['name']}")
print(f"Address: {place_details['result']['formatted_address']}")
print(f"Phone Number: {place_details['result'].get('formatted_phone_number', 'N/A')}")
print(f"Website: {place_details['result'].get('website', 'N/A')}")


Name: Hôtel Parc Belle-Vue
Address: 5 Av. Marie-Thérèse, 2132 Hollerich Luxembourg
Phone Number: 45 61 41 1
Website: https://www.goereshotels.com/belle-vue-en/


In [7]:
# Connect to DuckDB
# con = duckdb.connect('places.db')
# df = con.execute('SELECT * FROM places').fetchdf()

In [3]:
conn = duckdb.connect('../../data/raw/database.db')
%sql conn --alias duckdb

In [4]:
%%sql
select column_name from  information_schema.columns
where table_name = 'places';

Running query in 'duckdb'

column_name
place_id
name
vicinity
latitude
longitude


In [5]:
%%sql
select * from geonames
limit 10

Running query in 'duckdb'

adminCode,longitude,geonameId,toponymName,countryId,fcl,population,countryCode,name,fclName,adminCodes,countryName,fcodeName,adminName,latitude,fcode
ES,6.01278018951416,2960102,Schifflange,2960313.0,P,8155.0,LU,Schifflange,"city, village,...",{'ISO3166_2': ES},Luxembourg,seat of a third-order administrative division,Esch-sur-Alzette,49.50638961791992,PPLA3
VD,6.208889961242676,2960021,Vianden,2960313.0,P,1626.0,LU,Vianden,"city, village,...",{'ISO3166_2': VD},Luxembourg,seat of a second-order administrative division,Vianden,49.935001373291016,PPLA2
ES,6.042220115661621,2960042,Tétange,2960313.0,P,2929.0,LU,Tétange,"city, village,...",{'ISO3166_2': ES},Luxembourg,populated place,Esch-sur-Alzette,49.475830078125,PPL
ES,5.893060207366943,2960228,Niedercorn,2960313.0,P,3164.0,LU,Niedercorn,"city, village,...",{'ISO3166_2': ES},Luxembourg,section of populated place,Esch-sur-Alzette,49.536109924316406,PPLX
LU,6.130000114440918,2960250,Müllendorf,2960313.0,P,1024.0,LU,Müllendorf,"city, village,...",{'ISO3166_2': LU},Luxembourg,populated place,Luxembourg,49.68027877807617,PPL
ES,6.065279960632324,2960335,Leudelange,2960313.0,P,1924.0,LU,Leudelange,"city, village,...",{'ISO3166_2': ES},Luxembourg,seat of a third-order administrative division,Esch-sur-Alzette,49.568328857421875,PPLA3
LU,6.142020225524902,2960472,Heisdorf,2960313.0,P,1586.0,LU,Heisdorf,"city, village,...",{'ISO3166_2': LU},Luxembourg,populated place,Luxembourg,49.67206954956055,PPL
CA,5.9096999168396,2960485,Hautcharage,2960313.0,P,1495.0,LU,Hautcharage,"city, village,...",{'ISO3166_2': CA},Luxembourg,populated place,Capellen,49.574989318847656,PPL
LU,6.153890132904053,2960584,Fentange,2960313.0,P,1302.0,LU,Fentange,"city, village,...",{'ISO3166_2': LU},Luxembourg,populated place,Luxembourg,49.56277847290039,PPL
CL,6.031390190124512,2960684,Clervaux,2960313.0,P,1103.0,LU,Clervaux,"city, village,...",{'ISO3166_2': CL},Luxembourg,seat of a second-order administrative division,Clervaux,50.05472183227539,PPLA2


In [6]:
%%sql
show tables;
# show placedetail;

Running query in 'duckdb'

name
geonames
place_details
places


In [14]:
%%sql

select * from place_details where website = 'N/A'

Running query in 'duckdb'

place_id,name,address,phoneNumber,website
ChIJ5zyJKMRIlUcREbq-zdfRACY,Bonnevoie,2543 Bonnevoie Luxembourg,N/A,N/A
ChIJ_zmShNpIlUcRktD5HEzEJPk,Apart1hotel,"3 Rue d'Épernay, 1490 Gare Luxembourg",48 52 52,N/A
ChIJocM80ZRIlUcRxxjAWnNIgJg,Hôtel-Restaurant Chez Anna et Jean,"248 Rte de Thionville, 2610 Howald Hesperange, Luxembourg",48 21 69,N/A
ChIJn58nGChJlUcRVB4IkwnibaE,Hôtel Piemont,"56 Rte d'Esch 58, 1470 Hollerich Luxembourg",25 42 01 1,N/A
ChIJQ6jdtSxPlUcRUJtbl6OTXoQ,Guillaume Suites,"24 Rue du Cure, 1368 Ville-Haute Luxembourg",26 97 62 88,N/A
ChIJFwSkm-A0lUcRiTjKcG4oLTE,Eximport SA,"3-20 Rue Caspar-Mathias Spoo, 4323 Esch-sur-Alzette, Luxembourg",53 14 50,N/A
ChIJh_lifSkllUcRd8kaRBqNr6c,Thionville,"57100 Thionville, France",N/A,N/A
ChIJLcN7meA0lUcRtaBDivzB56g,IAP Immobilier Sàrl,"2 Rue Caspar-Mathias Spoo, 4323 Esch-sur-Alzette, Luxembourg",53 19 40,N/A
ChIJn2-lyBo1lUcRnZ4RyRzvflM,Hosteria Gusto,"145 Rue de Luxembourg, 4221 Esch-sur-Alzette, Luxembourg",26 17 87,N/A
ChIJpRwWwOI0lUcROtfqiAjlwFc,Standard Rooms in Center Esch-Alzette,"44 Boulevard J-F Kennedy Esch-sur-Alzette Esch-sur-Alzette, 4071 Esch-sur-Alzette, Luxembourg",621 423 586,N/A


In [15]:
%%sql

select * from places

Running query in 'duckdb'

place_id,name,vicinity,latitude,longitude
ChIJQVrlNdNIlUcRa3v0FrkGQ-g,Hotel Vauban,"10 Place Guillaume II, Luxembourg",49.610939025878906,6.130640983581543
ChIJld6G1ClPlUcRbXoEwx9_WUs,Best Western Plus,"3 Avenue Victor Hugo, Luxembourg",49.61863327026367,6.124682426452637
ChIJydDhxCtJlUcR5peH7n21z_s,Key Inn Appart-Hôtels,"42 Rue Albert Ier, Luxembourg",49.60702133178711,6.117996692657471
ChIJRa7IM8xIlUcRhHTQl-qHjGw,Sofitel Luxembourg Le Grand Ducal,"35 Rue du Laboratoire, Luxembourg",49.60520553588867,6.137516021728516
ChIJtfDL09dOlUcRXrGfmsSYdq4,Domus Hotel,"37 Avenue Monterey, Luxembourg",49.61027145385742,6.125014781951904
ChIJ9eUmbdBIlUcROI0fItQ-iU4,Hotel Bristol,"11 Rue de Strasbourg, Luxembourg",49.60121154785156,6.131739139556885
ChIJPbh1USpIlUcRfNU4CkNDobY,ibis budget Luxembourg Sud,Rue de Turi,49.53246307373047,6.111510753631592
ChIJF0RByupIlUcRGSeVG5gFzm4,Restaurant-Pizzeria-Auberge La Véranda,175 Route de Thionville,49.58829116821289,6.142622470855713
ChIJVyzznc1IlUcREG0F0dbRAAQ,Luxembourg,Luxembourg,49.61162185668945,6.131934642791748
ChIJiVm9mNZFlUcR1lpH199XBOA,Hôtel Empire,"34 Place de la Gare, Luxembourg",49.59960174560547,6.1327948570251465


In [8]:
# con.close()
conn.close()

In [13]:
formatted_location = format_coordinates(49.611671447753906, 6.130000114440918)

In [14]:
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={formatted_location}&radius={radius}&key={API_KEY}"

response = requests.get(url)
places = response.json()

In [ ]:
places["results"]

In [17]:

@dataclass
class Place:
    place_id: str
    name: str
    vicinity: str
    latitude: float
    longitude: float

In [18]:
def convert_to_place(result: dict) -> Place:
    """
    Convert a Google Maps API result to a Place object.

    Args:
        result (dict): The result dictionary from the Google Maps API.

    Returns:
        Place: A Place object representing the place.
    """
    place_id = result["place_id"]
    name = result["name"]
    vicinity = result.get("vicinity", "N/A")
    latitude = result["geometry"]["location"]["lat"]
    longitude = result["geometry"]["location"]["lng"]
    return Place(place_id, name, vicinity, latitude, longitude)

In [25]:
def get_all_places(location: str, radius: int):
        """
        Fetch all places within a given radius of a location.

        Args:
            location (str): The location to search around, in the format "latitude,longitude".
            radius (int): The radius in meters to search within.

        Returns:
            List[Place]: A list of Place objects representing the nearby places.
        """
        all_places = []
        url = f"{base_url}/place/nearbysearch/json?location={location}&radius={radius}&key={api_key}"

        while url:
            try:
                response = requests.get(url)
                response.raise_for_status()
            except requests.exceptions.RequestException as e:
                print(f"Error in nearby search request: {e}")
                break

            places = response.json()
            all_places.extend(places["results"])
            next_page_token = places.get("next_page_token")
            if next_page_token:
                time.sleep(2)  # Add a delay before making the next request
                url = f"{base_url}/place/nearbysearch/json?pagetoken={next_page_token}&key={api_key}"
            else:
                url = None

        return [convert_to_place(result) for result in all_places]

In [27]:
places = get_all_places(formatted_location, 50000)